In [15]:
!pip install torchvision==0.16.0 torchaudio==2.1.0 torchdata==0.7.0 torchtext==0.16.0

import torch
import torchvision

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

print("pytorch", torch.__version__)
print("torchvision", torchvision.__version__)


pytorch 2.1.0+cu121
torchvision 0.16.0+cu121


In [16]:
import numpy as np
from torchvision import tv_tensors
from torchvision.datasets.vision import VisionDataset
from torchvision.datasets.utils import download_and_extract_archive, verify_str_arg
from torchvision.ops import box_area

from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import collections
import os
from xml.etree.ElementTree import Element as ET_Element

try:
    from defusedxml.ElementTree import parse as ET_parse
except ImportError:
    from xml.etree.ElementTree import parse as ET_parse
from typing import Any, Callable, Dict, List, Optional, Tuple


class FracAtlasDetection(VisionDataset):
    """FracAtlas dataset."""

    def __init__(
        self,
        root: str,
        image_set: str = "train",
        # download: bool = False,
        transform: Optional[Callable] = None,
        target_transform: Optional[Callable] = None,
        transforms: Optional[Callable] = None,
    ):
        super().__init__(root, transforms, transform, target_transform)

        valid_image_sets = ["train", "test"]
        self.image_set = verify_str_arg(image_set, "image_set", valid_image_sets)

        self.url = "https://figshare.com/ndownloader/files/41725659"
        self.filename = "fracatlas.zip"

        # if download:
        if not os.path.isdir("data/FracAtlas"):
            os.makedirs("data", exist_ok=True)
            download_and_extract_archive(self.url, os.path.dirname(self.root), filename=self.filename, remove_finished=True)
            for subdir in ["Fractured", "Non_fractured"]:
              dirpath = os.path.join(self.root, "images")
              subdirpath = os.path.join(dirpath, subdir)
              for f in os.listdir(subdirpath):
                if not f.lower().endswith(".jpg"): continue
                os.rename(os.path.join(subdirpath, f), os.path.join(dirpath, f))
              os.rmdir(subdirpath)
        if not os.path.isdir(self.root):
            raise RuntimeError(
                "Dataset not found or corrupted. You can use download=True to download it"
            )

        image_dir = os.path.join(root, "images")
        target_dir = os.path.join(root, "Annotations", "PASCAL VOC")
        all_images = [os.path.splitext(x)[0] for x in os.listdir(image_dir)]

        # remove images with less than one fracture
        all_images = [x for x in all_images if len(self.parse_voc_xml(ET_parse(os.path.join(target_dir, x + ".xml")).getroot())["annotation"]["object"]) > 0]

        print(len(all_images))

        # 90% of images in train, in which 80% are actually training and 20% validation, and the last 10% in test
        file_names = []
        if image_set == "train":
            file_names = all_images[: int(0.9 * len(all_images))]
        elif image_set == "test":
            file_names = all_images[int(0.9 * len(all_images)) :]

        self.images = [os.path.join(image_dir, x + ".jpg") for x in file_names]
        self.targets = [os.path.join(target_dir, x + ".xml") for x in file_names]

        assert len(self.images) == len(self.targets)

    @property
    def annotations(self) -> List[str]:
        return self.targets

    def __getitem__(self, index: int) -> Tuple[Any, Any]:
        """
        Args:
            index (int): Index

        Returns:
            tuple: (image, target) where target is a dictionary of the XML tree.
        """
        img = Image.open(self.images[index]).convert("RGB")
        img = tv_tensors.Image(img)
        voc_dict = self.parse_voc_xml(ET_parse(self.annotations[index]).getroot())
        target = self.voc_dict_to_target(index, voc_dict, img)
        # print(target)

        if self.transform is not None:
            img, target = self.transform(img, target)

        return img, target

    def __len__(self) -> int:
        return len(self.images)

    @staticmethod
    def parse_voc_xml(node: ET_Element) -> Dict[str, Any]:
        voc_dict: Dict[str, Any] = {}
        children = list(node)
        if children:
            def_dic: Dict[str, Any] = collections.defaultdict(list)
            for dc in map(FracAtlasDetection.parse_voc_xml, children):
                for ind, v in dc.items():
                    def_dic[ind].append(v)
            if node.tag == "annotation":
                def_dic["object"] = [def_dic["object"]]
            voc_dict = {
                node.tag: {
                    ind: v[0] if len(v) == 1 else v for ind, v in def_dic.items()
                }
            }
        if node.text:
            text = node.text.strip()
            if not children:
                voc_dict[node.tag] = text
        return voc_dict

    def voc_dict_to_target(self, index: int, item, img):
        width = int(item["annotation"]["size"]["width"])
        height = int(item["annotation"]["size"]["height"])

        num_fractures = len(item["annotation"]["object"])

        boxes = [
            (
                int(obj["bndbox"]["xmin"]),
                int(obj["bndbox"]["ymin"]),
                int(obj["bndbox"]["xmax"]),
                int(obj["bndbox"]["ymax"]),
            )
            for obj in item["annotation"]["object"]
        ]

        boxes = torch.as_tensor(boxes, dtype=torch.float32).reshape(-1, 4)

        # extract labels
        labels = torch.ones((num_fractures,), dtype=torch.int64)

        image_id = index
        area = box_area(boxes)

        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_fractures,), dtype=torch.int64)

        target = {}
        target["boxes"] = tv_tensors.BoundingBoxes(boxes, format="xyxy", canvas_size=img.shape[-2:])
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        return target


In [17]:
# import shutil
# shutil.rmtree("data")
# os.mkdir("data")


In [18]:
from torchvision.transforms import v2 as T

def get_transform(train):
    transforms = []
    # transforms.append(T.ToImage())
    # if train:
    #     # transforms.append(T.RandomPhotometricDistort())
    #     # transforms.append(T.RandomZoomOut(fill={tv_tensors.Image: (0,0,0), "others": 0}))
    #     transforms.append(T.RandomHorizontalFlip(0.5))
    # transforms.append(T.ClampBoundingBoxes())
    # transforms.append(T.SanitizeBoundingBoxes())
    transforms.append(T.ToDtype(dtype=torch.float, scale=True))
    return T.Compose(transforms)


In [19]:
# os.makedirs("lib", exist_ok=True)
if not os.path.exists("engine.py"):
    os.system(
        "wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/engine.py -O engine.py"
    )
if not os.path.exists("utils.py"):
    os.system(
        "wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/utils.py -O utils.py"
    )
if not os.path.exists("transforms.py"):
    os.system(
        "wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/transforms.py -O transforms.py"
    )
if not os.path.exists("coco_utils.py"):
    os.system(
        "wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_utils.py -O coco_utils.py"
    )
if not os.path.exists("coco_eval.py"):
    os.system(
        "wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_eval.py -O coco_eval.py"
    )

# you also need to tweak them a bit to work with relative imports


In [20]:
from torchvision.models.detection import ssdlite320_mobilenet_v3_large, SSDLite320_MobileNet_V3_Large_Weights

from utils import collate_fn

model = ssdlite320_mobilenet_v3_large(weights=SSDLite320_MobileNet_V3_Large_Weights.DEFAULT)
dataset = FracAtlasDetection("data/FracAtlas", image_set="train", transform=get_transform(train=True))
data_loader = torch.utils.data.DataLoader(
  dataset,
  batch_size=2,
  shuffle=True,
  # num_workers=4,
  collate_fn=collate_fn,
)

# For Training
images, targets = next(iter(data_loader))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
output = model(images, targets)  # Returns losses and detections
print(output)

# For inference
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)  # Returns predictions
print(predictions[0])


717
{'bbox_regression': tensor(6.1969, grad_fn=<DivBackward0>), 'classification': tensor(6.9330, grad_fn=<DivBackward0>)}
{'boxes': tensor([[ 11.7742,   6.7620, 392.0815, 295.4871],
        [ 13.0175,   3.9538, 389.6967, 297.4302],
        [ 10.9037,  12.4899, 386.4312, 292.7108],
        ...,
        [239.4524, 184.4012, 251.1608, 194.0376],
        [257.9152,  97.8513, 274.0744, 119.3798],
        [185.8267,  65.6441, 205.6906,  76.4117]], grad_fn=<StackBackward0>), 'scores': tensor([0.0562, 0.0444, 0.0248, 0.0241, 0.0226, 0.0222, 0.0221, 0.0218, 0.0211,
        0.0209, 0.0207, 0.0207, 0.0202, 0.0199, 0.0199, 0.0198, 0.0198, 0.0197,
        0.0194, 0.0190, 0.0188, 0.0188, 0.0188, 0.0187, 0.0187, 0.0186, 0.0186,
        0.0185, 0.0185, 0.0184, 0.0184, 0.0183, 0.0183, 0.0183, 0.0183, 0.0182,
        0.0182, 0.0182, 0.0182, 0.0182, 0.0182, 0.0181, 0.0180, 0.0179, 0.0178,
        0.0178, 0.0177, 0.0177, 0.0176, 0.0176, 0.0175, 0.0175, 0.0174, 0.0173,
        0.0173, 0.0172, 0.0172, 0.017

In [21]:
from engine import train_one_epoch, evaluate

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("device:", device)

# use our dataset and defined transformations
dataset = FracAtlasDetection('data/FracAtlas', image_set="train", transform=get_transform(train=True))
dataset_test = FracAtlasDetection('data/FracAtlas', image_set="test", transform=get_transform(train=False))

# split the dataset in train and test set
# indices = torch.randperm(len(dataset)).tolist()
# dataset = torch.utils.data.Subset(dataset, indices[:-80])
# dataset_test = torch.utils.data.Subset(dataset_test, indices[-20:])

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=3,
    shuffle=True,
    # num_workers=4,
    collate_fn=collate_fn,
)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=1,
    shuffle=False,
    # num_workers=4,
    collate_fn=collate_fn,
)

model = ssdlite320_mobilenet_v3_large(weights=SSDLite320_MobileNet_V3_Large_Weights.DEFAULT)

# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
    params,
    lr=0.005,
    momentum=0.9,
    weight_decay=0.0005
)

# and a learning rate scheduler
# lr_scheduler = torch.optim.lr_scheduler.StepLR(
#     optimizer,
#     step_size=3,
#     gamma=0.1
# )

# let's train it for 5 epochs
num_epochs = 30

for epoch in range(num_epochs):
    model.eval()
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    # lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

print("That's it!")

print("Saving")
torch.save(model, 'fracatlas_fasterrcnn-mobilenetv3_v2.pth')
print("Saved")


device: cuda
717
717
Epoch: [0]  [  0/215]  eta: 0:00:35  lr: 0.000028  loss: 13.1996 (13.1996)  bbox_regression: 6.3516 (6.3516)  classification: 6.8480 (6.8480)  time: 0.1651  data: 0.0132  max mem: 688
Epoch: [0]  [ 10/215]  eta: 0:01:03  lr: 0.000262  loss: 13.1996 (13.1253)  bbox_regression: 6.0817 (5.9746)  classification: 6.8480 (7.1507)  time: 0.3077  data: 0.1496  max mem: 688
Epoch: [0]  [ 20/215]  eta: 0:00:54  lr: 0.000495  loss: 10.5051 (11.7597)  bbox_regression: 5.9874 (6.1055)  classification: 4.9404 (5.6542)  time: 0.2872  data: 0.1567  max mem: 688
Epoch: [0]  [ 30/215]  eta: 0:00:46  lr: 0.000729  loss: 9.3707 (10.5024)  bbox_regression: 5.8488 (5.6768)  classification: 3.3119 (4.8255)  time: 0.2235  data: 0.1177  max mem: 688
Epoch: [0]  [ 40/215]  eta: 0:00:43  lr: 0.000962  loss: 7.1711 (9.7065)  bbox_regression: 4.3655 (5.3680)  classification: 2.8750 (4.3385)  time: 0.2116  data: 0.0938  max mem: 688
Epoch: [0]  [ 50/215]  eta: 0:00:37  lr: 0.001195  loss: 7.402